### 2乗和誤差 (mean squared error)
$$ E = \frac {1}{2} \sum_{k} (y_k - t_k)^2$$

- $ y_k $はニューラルネットワークの出力
- $ t_k $は教師データ
- $ k $は データの次元数 

In [3]:
import numpy as np

y = [0.1, 0.05, 0.6, 0.0,0.05,0.1,0.0,0.1,0.0,0.0] # ニューラルネットワークの出力(ソフトマックス関数の出力なので確率)
t = [0,0,1,0,0,0,0,0,0,0] # 教師データ(正解となるラベルを1、それ以外を0とする)

# 2乗和誤差
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

# 例1
mean_squared_error(np.array(y), np.array(t))

0.097500000000000031

In [10]:
# 7の確率が最も高い場合
y = [0.1, 0.05, 0.1, 0.0,0.05,0.1,0.0,0.6,0.0,0.0]
# 例2
mean_squared_error(np.array(y), np.array(t))

0.59750000000000003

一つ目の結例の損失関数の方が小さくなっており、教師データとの誤差が小さいことがわかります。
つまり、一つ目の例のほうが、出漁結果が教師データにより適合していることを2乗和誤差は示しているのです。

### 交互エントロピー誤差(cross entropy error)
$$ E = \sum_{k} t_k \log y_k $$

- $ \log $は底が e の自然対数( $\log e$ ) を表します
- $ y_k$ はニューラルネットワークの出力 
- $ t_k$ は正解ラベル（正解となるインデックスだけが1でその他は0）

In [4]:
import numpy as np

# エントロピー誤差の実装
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

t = [0,0,1,0,0,0,0,0,0,0]
y = [0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0]
cross_entropy_error(np.array(y), np.array(t))

0.51082545709933802

In [5]:
y = [0.1, 0.05,0.1,0.0,0.05,0.1,0.0,0.6,0.0,0.0]
cross_entropy_error(np.array(y), np.array(t))

2.3025840929945458